## Step 1: Mount Google Drive and Clone Repository

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os
os.chdir('/content')

# Clone the glue-factory repository
!git clone https://github.com/Dilshan-Nuwantha/glue-factory.git
os.chdir('/content/glue-factory')

## Step 2: Install Dependencies

In [ ]:
# Install required packages
!pip install -e . -q
!pip install torch torchvision -q

## Step 3: Create 100-Sample Config

In [ ]:
# Create config for 100 samples
config_content = """data:
    name: homographies
    data_dir: revisitop1m
    train_size: 100
    val_size: 10
    batch_size: 32
    num_workers: 2
    homography:
        difficulty: 0.7
        max_angle: 45
    photometric:
        name: lg
model:
    name: two_view_pipeline
    extractor:
        name: gluefactory_nonfree.superpoint
        max_num_keypoints: 512
        force_num_keypoints: True
        detection_threshold: 0.0
        nms_radius: 3
        trainable: False
    ground_truth:
        name: matchers.homography_matcher
        th_positive: 3
        th_negative: 3
    matcher:
        name: matchers.lightglue
        filter_threshold: 0.1
        flash: false
        checkpointed: true
train:
    seed: 0
    epochs: 10
    log_every_iter: 10
    eval_every_iter: 50
    lr: 1e-4
    lr_schedule:
        start: 5
        type: exp
        on_epoch: true
        exp_div_10: 5
benchmarks:
    hpatches:
      eval:
        estimator: opencv
        ransac_th: 0.5
"""

with open('gluefactory/configs/superpoint+lightglue_homography_100samples.yaml', 'w') as f:
    f.write(config_content)

print("Config created successfully!")

## Step 4: Set Up Data Directory

In [ ]:
# Create data directories
!mkdir -p data/revisitop1m
!mkdir -p outputs/training

print("Data directories created!")

## Step 5: Start Training with 100 Samples

In [ ]:
# Run training
!python -m gluefactory.train colab_test_100 \
    --conf gluefactory/configs/superpoint+lightglue_homography_100samples.yaml \
    data.data_dir=/content/glue-factory/data/revisitop1m

## Step 6: Download Results to Google Drive

In [ ]:
import shutil

# Copy trained model to Google Drive
output_path = '/content/glue-factory/outputs/training'
gdrive_path = '/content/gdrive/My Drive/glue-factory-results'

!mkdir -p '{gdrive_path}'
!cp -r '{output_path}' '{gdrive_path}/'

print(f"Results saved to Google Drive: {gdrive_path}")

## Optional: Monitor Training Progress

In [ ]:
# Check outputs
import os
output_dir = '/content/glue-factory/outputs/training'
if os.path.exists(output_dir):
    for root, dirs, files in os.walk(output_dir):
        level = root.replace(output_dir, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f'{indent}{os.path.basename(root)}/')
        subindent = ' ' * 2 * (level + 1)
        for file in files[:10]:  # Show first 10 files
            print(f'{subindent}{file}')